# **Feature Selection**
Here we explore methodologies to identify which features are useful provide a higher predictive power to the model. Given a dataset, a model trained on it can depend on features directly on derived features. How do we tell wich features are the most useful? Multiple approaches exist, which are based on simple ideas of univariate analysis to complex multivariate analysis. In univariate analysis we look at how a single feature contribute to the model. Although useful, it does have pitfalls as some features are better together. In multivariate analysis we can tell which features perform well and more importantly which perform well together. Various techniques exist driven differentiated by how information is extracted. When data contains label like the case here, we use supervised techniques, nevetheless, unsupervised techniques can be used for unlabelled data.

Collaborative filtering is built on the assumption that a good way to predict the
preference of an active consumer for a target product is to find other consumers
who have similar preferences and use their votes for that product to make a
prediction.
As noted in the [source page](https://www.analyticsvidhya.com/blog/2020/10/feature-selection-techniques-in-machine-learning/), these techniques can be classified as follows
- **Filter methods:** based on features properties highlighted via univariate analysis

- **Wrapper methods:** With a specific learning algorithm, these methose can perform a greedy search of the best feature by fitting models with possible subsets of features, assessing their quality by learning and evaluating a classifier with that feature subset. 
- **Embedded methods:** Here they aim to combine the power of both filters and wrapper while maintaining reasonable computational cost.
- **Hybrid method:** Hybrid methods basically select features via a global transformation reduces the data to a desided number of dimensions. The new features can bear little or no resemblance to the initial features.



In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Get some toy data

In [ ]:
with open(r'C:path', 'rb') as f:
    df_application = pickle.load(f)

In [ ]:
pd.options.display.max_columns = None
df_application.head(2)

In [ ]:
from optbinning import OptimalBinning, BinningProcess

In [ ]:
# format columns to lower case (just for a nice look :) )
df_application.columns = [col.lower() for col in df_application.columns]
df_application.head(2)

Instantiate the `BinningProcess` with feature names and the fit

In [ ]:
select_cols = df_application.columns[1:].to_list()
binning_process = BinningProcess(select_cols)
binning_process.fit(df_application[select_cols], df_application.target)
binning_table = binning_process.summary()
binning_table

In [ ]:
binning_table[binning_table['name']=='ext_source_3']

Build a function that gets the summary table for Information Values, Jensen-Shannon entropy, Gini and quality.

In [ ]:
def get_metrics(x, y):
    select_cols = x.columns.to_list()
    binning_process = BinningProcess(select_cols)
    binning_process.fit(x, y)
    binning_table = binning_process.summary()
    binning_table.sort_values(by='iv', inplace=True, ascending=False)
    binning_table['interpretation'] = binning_table['iv'].apply(interpretation)
    return binning_table

def interpretation(iv):
    if iv < 0.02:
        return 'useless'
    elif iv < 0.1:
        return 'weak'
    elif iv < 0.3:
        return 'medium'
    elif iv < 0.5:
        return 'strong'
    else:
        return 'suspicious'

In [ ]:
binning_table_metrics = get_metrics(df_application[select_cols], df_application.target)

In [ ]:
binning_table_metrics.head(10)

In [ ]:
binning_table_metrics.tail(10)

In [ ]:
binning_table_metrics[binning_table_metrics['interpretation'].isin(['strong', 'medium'])]

### 2.2. Variance threshold
Here we remove the data with smaller variance. For simplicity we will start using just numerical data

In [ ]:
from sklearn import feature_selection

In [ ]:
#working with numerical data
X = df_application.drop('target', axis=1)
Y = df_application.target
numerical_columns = X.select_dtypes(include=np.number).columns.values

In [ ]:
constant_threshold = feature_selection.VarianceThreshold(threshold=0.001)
constant_threshold.fit(X[numerical_columns])

In [ ]:
#reduced features is
# df_application_train, df_application_test, y_train, y_test
X_filter = constant_threshold.transform(X[numerical_columns])
# X_tfilter = constant_threshold.transform(df_application_test[numerical_columns])

In [ ]:
# X[numerical_columns].columns[constant_threshold.get_support()]

### 2.3 Information gain
Here we will use the mutual information to streamline the features

In [ ]:
select_features = X[numerical_columns].columns[constant_threshold.get_support()]

In [ ]:
importance = feature_selection.mutual_info_classif(X[select_features].fillna(X[select_features].mean()), Y)

In [ ]:
feature_importances = pd.Series(importance, select_features)

In [ ]:
feature_importances.sort_values(ascending=False).head(10).plot(kind='barh', color='teal')

Scikit-learn provides functionality to automatically select features when a measure and selection criteria are provided. In this case, we can use selection pipeline and metrics like `Percentile`, or top best, to select a particular number of columns. Scikit-learn untitilites for this includes
- [`SelectPercentile`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile)
- [`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)

These can be used with measures like mutual information (`mutual_info_classif`), Chi Square (`chi2`), Fisher information etc. We will demonstrate for mutual information. Note that this takes a long time to run.

In [ ]:
# sel = feature_selection.SelectPercentile(
#     feature_selection.mutual_info_classif,
#     percentile=10
# ).fit(X_filter, Y)
# X_filter.columns[sel.get_support()]

### 2.4 AUC
AUC is good measure for model performance for various reasons. Here we want to use AUC to measure the performance of a model build on a single feature. At the end we select features with high AUC.

In [ ]:
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_application = X[select_features].fillna(X[select_features].mean())
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y)

In [ ]:
roc_auc = []
for feature in numerical_columns:
    clf = LogisticRegression(max_iter=100, random_state=42)
    clf.fit(X[feature].fillna(0).values.reshape(-1, 1), Y)
    y_pred = clf.predict(X_test[feature].fillna(0).values.reshape(-1, 1))
    roc_auc.append(metrics.roc_auc_score(y_test, y_pred))

In [ ]:
roc_auc_series = pd.Series(roc_auc, index=numerical_columns).sort_values(ascending=False)
roc_auc_series.head()

Any feature with AUC < 0.5 are not useful.

In [ ]:
roc_auc_series[roc_auc_series>0.5]

We can then used this to build a model

In [ ]:
def run_logreg(X_train, y_train, X_test, y_test):
    clf =  LogisticRegression(C=3, max_iter=100, random_state=42)
    clf.fit(X_train.fillna(0), y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy on test set is: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
%% time
s = roc_auc_series[roc_auc_series>0.5].index.to_list()
run_logreg(X[numerical_columns], Y)

### 2.5 Correlation coefficients.
This can be a quick and easy way to see which features are correlated with the target. Correlation compute the Perason Correlation, the logic behind its used for feature selection is that the good variables are highly correlated with the target

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# For ease of plotting we will only select 10 features
correlation_matrix = X[numerical_columns[:10]].merge(Y, right_index=True, left_index=True).corr()

In [ ]:
#plotting heatmap
plt.figure(figsize=(10,6))
sns.heatmap(correlation_matrix, annot=True)

## 3 Wrapper Methods:
Wrappers require some method to search the space of all possible subsets of features, assessing their quality by learning and evaluating a classifier with that feature subset. The feature selection process is based on a specific machine learning algorithm that we are trying to fit on a given dataset. It follows a greedy search approach by evaluating all the possible combinations of features against the evaluation criterion. The wrapper methods usually result in better predictive accuracy than filter methods.
 - Forward Feature Selection
 - Backward Feature Elimination
 - Exhaustive Feature Selection
 - Recursive Feature Elimination
 
 Forward Feature Selection: Starts with the best and gradually adds the others
 Backward Feature Elimination: Starts will all and start elimicating, with worst the first to get out
 Exhaustive Feature Selection: Brute force method that searches each of the possible combinations
 
 We are going to address each of the methods, but for now will start with Recursive Feature Elimination as the others needs mlxtend library

### 3.1 Recursive Feature Elimination

First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute.

Then, the least important features are pruned from the current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

In [ ]:
# Recursive Feature Elimination
rfe = feature_selection.RFE(
    LogisticRegression(C=3, max_iter=1000, random_state=42),
    n_features_to_select=10
)
rfe.fit(X[numerical_columns].fillna(X[numerical_columns].mean()), Y)

In [ ]:
rfe.get_feature_names_out()

## 4 Embedded Methods:
The challenged we had in previous methods is that filters word with one feature, and wrapper selects a group of feature and then train the model from start to finish. The question is what if some features only present their power in some parameter space. Embedded methods are iterative in the sense that takes care of each iteration of the model training process and carefully extracts those features which contribute the most to the training for a particular iteration. [Read more here](https://www.analyticsvidhya.com/blog/2020/10/feature-selection-techniques-in-machine-learning/)

### 4.1 LASSO Regularization
Lasso methods applies regularization which can pernalize a feature differently and at different stages of the training interation process. Here we will use a `SelectFromModel` class. 

In [ ]:
X_t = X[numerical_columns].fillna(X[numerical_columns].mean())
lrg = LogisticRegression(C=1, penalty='l1', solver='liblinear', max_iter=1000, random_state=42)
model = feature_selection.SelectFromModel(
    lrg.fit(X_t, Y),
    prefit=True
)
X_new = model.transform(X_t)

In [ ]:
X_new.shape

In [ ]:
X_t.shape

### 4.2 Random Forest Importance
Random Forest is an ensemble methods that builds many trees and a prediction is made by a concensus aggrement by multiple trees. The performance is measured by how well they improve the purity of the node using Gini impurity.

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_num, Y)

In [ ]:
#get the importance of the resulting features
rf_imporatance_series = pd.Series(
    rf.feature_importances_, 
    X_num.columns
    ).sort_values(ascending=False)
rf_imporatance_series

In [ ]:
rf_imporatance_series.head()

## 5 Global Methods:
 - Shapely values

### 5.1 SHAP Values
SHAP Values (an acronym from SHapley Additive exPlanations) help in breaking down a prediction into components that illustrate the impact of each feature. This could be granular at the level of each instance (single row of data) or aggregated to see the models breakdown according to each feature. For some explanation see [here](https://www.kaggle.com/dansbecker/shap-values).

In [ ]:
import numpy as np
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Select numberical features
feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.int64]]
df_application = data[feature_names].fillna(0)
train_X, val_X, train_y, val_y = train_test_split(df_application.drop('TARGET', axis=1), df_application['TARGET'], random_state=1)
model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

In [ ]:
#Now that we have a model, let's look at the prediction for a single row. Let's use row 3
row_to_check = 3
data_for_prediction = val_X.iloc[row_to_check:row_to_check+1]
model.predict_proba(data_for_prediction)

In [ ]:
val_X.iloc[row_to_check:row_to_check+1]

This customer is 91% likely to be a good customer according to the model. Now let's see is the main driver behind the prediction

In [ ]:
# Create object that can calculate shap values
explainer = shap.TreeExplainer(model)
# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)

In [ ]:
shap_values

The shap values above is a 2D array. First is the values for negtive outcome (bad customer) and the second the values for the positive outcome ( good  customer). Since we are looking at prediction of negative outcome we look at the second array

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

In [ ]:
explainer2 = shap.Explainer(model)
shap_values2 = explainer2(val_X.head(50))

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values2[0])

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values2[:,"RM"], color=shap_values2)

In the explainer we us <code> shap.TreeExplainer() </code> as we have a tree. This is optimized to work for tree-base algorithms. We can use other specialised explainer can be view in the [official shap page](https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html), including <code> shap.Linear()</code>.

__Global Interpretation__:

In the demonstration above, we showed a break down for an individual. Now, let's look at what happens when we aggregate to the model level. Basically we waht to how what is driving the overall model performance (feature importance)

In [ ]:
%%time
shap_values = explainer.shap_values(val_X.head(50))

In [ ]:
# Make plot for top 10.
shap.summary_plot(shap_values[1], val_X.head(50))

The plot is made of many dots, each dot has three characteristics:

- Vertical location shows what feature it is depicting
- Color shows whether that feature was high or low for that row of the dataset
- Horizontal location shows whether the effect of that value caused a higher or lower prediction.

For example, the point in the upper left is for an individual with few days at worm, reducing the prediction by 0.05.
The plot visibily shows that:
- The model ignored flag_document_14.
- Higer values of days_birth caused higher predictions and lower values lower predictions

According to SHAP the top four features are 
- days_birth
- days_employed
- days_in_publish
- flag_document_3


In [ ]:
# # Using kernel explainer
# k_explainer = shap.KernelExplainer(model.predict_proba, train_X)
# # Calculate Shap values
# k_shap_values = k_explainer.shap_values(data_for_prediction)
# shap.force_plot(k_explainer.expected_value[1], k_shap_values[1], data_for_prediction)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Decision Tree")
tree_disp = PartialDependenceDisplay.from_estimator(model, val_X.head(50), ["DAYS_BIRTH", "DAYS_EMPLOYED"], ax=ax)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit_transform(data['AMT_INCOME_TOTAL'].head())

In [ ]:
data.head()

## 6 Hybrid methods
As mentioned, these are methods that transforms the data into a completely different vector space and bear little or no resemblance to the original data yet carry the same information. They are commonly referred to as dimentionalily reduction methods 

In [ ]:
df_application.head()

In [ ]:
%run ../src/data_utils.py

In [ ]:
dataset1 = mk.dataset1

In [ ]:
sess = saspy.SASsession(
        cfgfile=mk.saspy_file_path,
        cfgname=mk.saspy_cfgname
    )

In [ ]:
sess.saslib(dataset1['lib_name'], path=dataset1['path'])
df = sess.sd2df(dataset1['table_name'], libref=dataset1['lib_name'], method="CSV")

In [ ]:
df.head()

In [ ]:
df.columns.to_list()

In [ ]:
[col for col in df.columns if col.lower().find("date")>0]

In [ ]:
[col for col in df.columns if col.lower().find("exclusion")>0]

In [ ]:
df.columns[25:50]

In [ ]:
df.shape

In [ ]:
df.info(verbose=True, null_counts=True)

### LGD Data Descrete

In [ ]:
import numpy as np

In [ ]:
%run ../src/data_utils.py
dataset2 = mk.dataset2

In [ ]:
sess = saspy.SASsession(
        cfgfile=mk.saspy_file_path,
        cfgname=mk.saspy_cfgname
    )

In [ ]:
sess.saslib(dataset2['lib_name'], path=dataset2['path'])
lgd_data = sess.sd2df(dataset2['table_name'], libref=dataset2['lib_name'], method="CSV")

In [ ]:
lgd_data.head(2)

Model Agnostic

In [ ]:
lgd_data.shape

In [ ]:
numerical_columns = lgd_data.select_dtypes(include=np.number).columns.values
categorical_columns = lgd_data.select_dtypes(include=['object', 'category']).columns.values

In [ ]:
import category_encoders as ce

In [ ]:
Encode Categorical variables. For now, lets used  TargetEncoder()

In [ ]:
encoder = ce.TargetEncoder()

In [ ]:
lgd_data_cat = encoder.fit_transform(lgd_data[categorical_columns], lgd_data['LGD_bad_ind'])

Feature Selection
Select the top N

In [ ]:
processed_data = lgd_data.copy()
processed_data[categorical_columns] = lgd_data_cat

In [ ]:
processed_data.head()

In [ ]:
from sklearn import feature_selection

In [ ]:
selector = feature_selection.VarianceThreshold(threshold=0.0025)
X_reduced = selector.fit_transform(processed_data.drop('LGD_bad_ind', axis=1), processed_data.LGD_bad_ind)
X_reduced.shape

In [ ]:
processed_data.shape

The function get_support can be used to generate the list of features that were kept.

In [ ]:
cols = selector.get_support(indices=True)
selected_columns = processed_data.iloc[:,cols].columns.tolist()

In [ ]:
len(selected_columns)

Same approach for mutual information

In [ ]:
def get_best_feature_importance(X, y, metric, k=5):

    selector = feature_selection.SelectKBest(metric)
    X_reduced = selector.fit_transform(X, y)
    cols = selector.get_support(indices=True)
    selected_columns = processed_data.iloc[:,cols].columns.tolist()

    return  selected_columns

In [ ]:
get_best_feature_importance(
    processed_data.drop('LGD_bad_ind', axis=1).head(100).fillna(0), 
    processed_data.LGD_bad_ind.head(100), 
    metric=feature_selection.mutual_info_classif, 
    k=5)

We note that 19 columns have been dropped. We can explore the dropped columns

In [ ]:
def _get_selected_columns(selector, X, y):
    cols = selector.get_support(indices=True)
    selected_columns = X.iloc[:,cols].columns.tolist()
    return selected_columns

In [ ]:
selector = feature_selection.SelectPercentile(feature_selection.mutual_info_classif, percentile=25)
X_reduced = selector.fit_transform(
    processed_data.drop('LGD_bad_ind', axis=1).head(100).fillna(0), 
    processed_data.LGD_bad_ind.head(100)
    )
X_reduced.shape

_get_selected_columns(
    selector, 
    processed_data.drop('LGD_bad_ind', axis=1).head(100).fillna(0), 
    processed_data.LGD_bad_ind.head(100)
    )

In [ ]:
selector = feature_selection.SelectKBest(feature_selection.f_classif, k=20)
X_reduced = selector.fit_transform(
    processed_data.drop('LGD_bad_ind', axis=1).head(100).fillna(0), 
    processed_data.LGD_bad_ind.head(100)
    )

X_reduced.shape

_get_selected_columns(
    selector, 
    processed_data.drop('LGD_bad_ind', axis=1).head(100).fillna(0), 
    processed_data.LGD_bad_ind.head(100)
    )

In [ ]:
lgd_data.info(verbose=True, null_counts=True)

In [ ]:
import pandas as pd
import numpy as np
import pickle
from optbinning import Scorecard, BinningProcess
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
with open('../data/lgd_data.pkl', 'wb') as f:
    pickle.dump(lgd_data, f)

In [ ]:
with open('../data/lgd_data.pkl', 'rb') as f:
    lgd_data = pickle.load(f)

In [ ]:
pd.options.display.max_columns = None
lgd_data.head()

Reduce dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    lgd_data.drop('LGD_bad_ind', axis=1), lgd_data.LGD_bad_ind, test_size=0.2, random_state=42, stratify=lgd_data.LGD_bad_ind)

In [ ]:
X_train.columns = [column.lower() for column in X_train.columns]
X_test.columns = [column.lower() for column in X_train.columns]

In [ ]:
categorical_features = df_application_train.select_dtypes(include=['object', 'category']).columns.values
numerical_features = df_application_train.select_dtypes(include=np.number).columns.values

### 6.1 Principal Component Analysis

In [ ]:
%matplotlib inline

# Ignore deprecated warning
import warnings
warnings.filterwarnings("ignore")

# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set font scale and style
plt.rcParams.update({'font.size': 15})

from sklearn.model_selection import train_test_split

Create features matrix

In [ ]:
X_train.head()

In [ ]:
# Feature matrix and class label
cols_to_drop = ['naics_industry_cd']
X, y = X_train.drop(cols_to_drop, axis = 1), y_train

Transformation pipeline
1. Impute missing values

In [ ]:
# Import custom classes
%run ../src/data_utils.py
%run ../src/imputer.py'
%run ../src/transforms.py

### 6.2 Singular Value Decomposition

In [ ]:
pwd

df.columns

### 6.3 Linear Discriminant Analysis

y = 'LGD_bad_ind'

In [ ]:
y = 'LGD_bad_ind'